## Loading up packages

In [44]:
import sys
sys.path.append('/home/keeganfl/Desktop/Work_Fall_2021/Fall_2021_Work_journal/tools/')

In [45]:
# import plastid
# data structure for mapping read alignments to genomic positions
from plastid import BAMGenomeArray, GenomeArray, VariableFivePrimeMapFactory, \
                        GTF2_TranscriptAssembler, GFF3_TranscriptAssembler, \
                        Transcript, ThreePrimeMapFactory, CenterMapFactory
import plastid
from plastid.plotting.plots import *
import numpy as np
import os
from Bio import SeqIO
import numpy
import math
import pandas as pd
from scipy import stats
import keegan_analysis_tools as kat
from statsmodels.nonparametric.smoothers_lowess import lowess
import warnings
import matplotlib.pyplot as plt
%matplotlib inline
import csv
from scipy.sparse.linalg import lsqr

In [46]:
# Define the path to our Bam files
data_path = "/home/keeganfl/Desktop/Work_Fall_2021/genomes_&_samples/mmus/"
csv_path = "/home/keeganfl/Desktop/Work_Fall_2021/data_tables/position_counts/mmus/"
save_path = "/home/keeganfl/Desktop/Work_Fall_2021/data_tables/elongation_rates/mmus/"
gtf_name = 'mm10.refGene.gtf'
mutation = 'control'
samp_num = '2'

## Loading up the data files 

In [47]:
# load the transcript annotations from the GTF file.
# GTF2_TranscriptAssembler returns an iterator, so here we convert it to a list.
transcripts = list(GTF2_TranscriptAssembler(open(data_path + gtf_name),return_type=Transcript))

## Getting the codon sequence:

For now I am deleting this section along with the normalization section since I believe that the raw rates will be sufficient once normalized. 

## Loading up the csv file.

In [48]:
data = kat.load_count_positions(mutation+"_RPF_"+samp_num+"_counts.csv",csv_path)

## Getting elongation rates for all genes

In [49]:
p_list = kat.calculate_p(data)

In [50]:
pbar_list = kat.calculate_pbar(p_list)

In [51]:
lbar_list = kat.calculate_lbar(pbar_list)

In [52]:
# calculate the scaled initiation and termination rates

init_r= []
for pbar in pbar_list:
    if pbar[0] == 0:
        init_r.append(1/(1-10*0.00001))
    else:
        init_r.append(1/(1-10*pbar[0]))

term_r = []
for p in p_list:
    if p[-1] ==0:
        term_r.append(1/0.00001)
    else:
        term_r.append(1/(p[-1]))


In [53]:
elongation_list = lbar_list

In [54]:
for elon, i in zip(elongation_list, list(range(len(elongation_list)))):
    elongation_list[i] = elon.tolist()

## Save as a csv

In [55]:
# Create a list that contains all of the gene_ids and transcript_ids of the transcripts
gene_id = []
transcript_id = []

for transcript in transcripts:
    gene_id.append(transcript.attr["gene_name"])
    transcript_id.append(transcript.attr["transcript_id"])

In [56]:
# Insert the gene ids and transcript ids into the codon_count list. 
for i,j in zip(elongation_list, range(len(gene_id))):
    i.insert(0,gene_id[j])
    i.insert(0,transcript_id[j])

In [57]:
# Calculate the longest cds region in our new list of counts
l_tr = kat.find_max_list(elongation_list)

# Define a header that includes labels for the transcript and gene ID as 
# well as numbers that index the cds region position.
header=["transcript_id","gene_id"]+list(range(l_tr))

# insert that header into our counts list. 
elongation_list.insert(0,header)

In [58]:
with open(save_path + mutation + '_' + samp_num + 'raw_elongation_rates', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(elongation_list)